In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time

%matplotlib inline

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist_data = input_data.read_data_sets('/tmp/data/', one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
n_classes = 10
feature_count = 28 * 28

In [5]:
n_units_hl = (500, 500, 500)
batch_size = 300
precision = 32

In [6]:
type_str = 'float{0}'.format(precision)
x = tf.placeholder(type_str, shape=(None, feature_count))
y = tf.placeholder(type_str)

In [7]:
print(x)
print(y)

Tensor("Placeholder:0", shape=(?, 784), dtype=float32)
Tensor("Placeholder_1:0", dtype=float32)


In [8]:
k_key_weights = 'weights'
k_key_biases = 'biases'
k_key_units = 'units'

In [9]:
def nn_model(data):
    layers = []
    real_n_units = list(n_units_hl)
    real_n_units.append(n_classes)
    for idx, n_units in enumerate(real_n_units):
        n = n_units
        m = feature_count
        if idx is not 0:
            m = n_units_hl[idx - 1]
        layer = {k_key_weights: tf.Variable(tf.random_normal([m, n])),
                 k_key_biases: tf.Variable(tf.random_normal([n]))}
        layers.append(layer)
    
    for idx, layer in enumerate(layers):
        if idx is 0:
            layer[k_key_units] = tf.add(tf.matmul(data, layer[k_key_weights]), layer[k_key_biases])
        else:
            layer[k_key_units] = tf.add(tf.matmul(layers[idx - 1][k_key_units], layer[k_key_weights]), layer[k_key_biases])
        
        if idx is not len(layers) - 1:
            layer[k_key_units] = tf.nn.relu(layer[k_key_units])
    
    return layers[-1][k_key_units]

In [10]:
def train_nn(data):
    prediction = nn_model(data)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10  # Number of iterations
    
    with tf.Session() as sess:
        start = time.time()
        sess.run(tf.global_variables_initializer())
        print('Initialized global variables...')
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist_data.train.num_examples/batch_size)):
                x_epoch, y_epoch = mnist_data.train.next_batch(batch_size)
                _, c_epoch = sess.run([optimizer, cost], feed_dict = {x: x_epoch, y: y_epoch})
                epoch_loss += c_epoch
            print('Epoch {0} completed out of {1} epochs, epoch_loss: {2}.'.format(epoch, hm_epochs, epoch_loss))
        print()
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy: {0:.2f}%'.format(accuracy.eval({x: mnist_data.test.images, y: mnist_data.test.labels}) * 100.0))
        end = time.time()
        print('Duration: {0:.2f}s'.format(end - start))

In [11]:
train_nn(x)

Initialized global variables...
Epoch 0 completed out of 10 epochs, epoch_loss: 928440.4586181641.
Epoch 1 completed out of 10 epochs, epoch_loss: 217513.58026123047.
Epoch 2 completed out of 10 epochs, epoch_loss: 135492.57049560547.
Epoch 3 completed out of 10 epochs, epoch_loss: 92363.11723327637.
Epoch 4 completed out of 10 epochs, epoch_loss: 66331.73112487793.
Epoch 5 completed out of 10 epochs, epoch_loss: 46589.691635131836.
Epoch 6 completed out of 10 epochs, epoch_loss: 33106.30202674866.
Epoch 7 completed out of 10 epochs, epoch_loss: 23185.222345352173.
Epoch 8 completed out of 10 epochs, epoch_loss: 16078.706481218338.
Epoch 9 completed out of 10 epochs, epoch_loss: 11841.0369079113.

Accuracy: 93.77%
Duration: 5.92s
